In [1]:
! du -sh ./*

52K	./LSTM.ipynb
3.5M	./data_analysis.ipynb
548K	./data_analysis_jingzhi.ipynb
40K	./feature_engineering.ipynb
4.0K	./notebook_template.ipynb
184K	./save_test.npz
236K	./test.ipynb
32K	./train-Copy1.ipynb
104K	./train.ipynb


In [1]:
import sys
import os
from time import time 
import pandas as pd
import numpy as np
from tqdm import tqdm
import multiprocessing



sys.path.append('../')
from utils import (
    check_columns, 
    check_category_column, 
    save_model, 
    transform_category_column,
    get_time_diff,
    load_model,
    LogManager,
    timer,
    standard_scale,
    log_scale,
    remove_cont_cols_with_small_std,
)
import conf
from mlpipeline.feature_engineering import (
    USING_LABEL,
    FAULT_LABEL,
)

In [2]:
df = pd.read_feather(os.path.join(conf.DATA_DIR, "test_file_03.feather"))

In [3]:
test_dict = dict(tuple(df.groupby(["serial_number", "model"])))

In [4]:
fault_df = df[df["flag"]==1]

In [5]:
fault_dic = dict(tuple(fault_df.groupby(["serial_number", "model"])))

In [6]:
fault_dic.keys()

dict_keys([('disk_103990', 1), ('disk_104617', 1), ('disk_108276', 1), ('disk_110185', 1), ('disk_113535', 1), ('disk_11374', 1), ('disk_116937', 1), ('disk_117550', 2), ('disk_11788', 2), ('disk_119072', 1), ('disk_123077', 2), ('disk_123598', 1), ('disk_125746', 1), ('disk_125953', 2), ('disk_126915', 2), ('disk_127609', 1), ('disk_128125', 2), ('disk_128986', 2), ('disk_131630', 2), ('disk_133408', 2), ('disk_13400', 1), ('disk_13593', 1), ('disk_135970', 1), ('disk_138123', 1), ('disk_138774', 1), ('disk_139345', 1), ('disk_139874', 1), ('disk_139880', 1), ('disk_141029', 2), ('disk_142247', 1), ('disk_143767', 1), ('disk_145360', 1), ('disk_15301', 2), ('disk_15938', 2), ('disk_19368', 1), ('disk_21342', 1), ('disk_21882', 1), ('disk_23869', 1), ('disk_2510', 2), ('disk_28293', 1), ('disk_30245', 1), ('disk_3306', 2), ('disk_3515', 2), ('disk_38051', 1), ('disk_4086', 1), ('disk_41015', 1), ('disk_45690', 2), ('disk_46746', 2), ('disk_47904', 2), ('disk_48529', 2), ('disk_49239', 

In [7]:
instance = test_dict[('disk_100420', 1)].set_index("dt")
instance

,serial_number,model,smart_1_normalized_min_7,smart_3_normalized_min_7,smart_7_normalized_min_7,smart_184_normalized_min_7,smart_187_normalized_min_7,smart_191_normalized_min_7,smart_195_normalized_min_7,smart_5raw_min_7,...,smart_191_normalized,smart_195_normalized,smart_5raw,smart_194raw,smart_197raw,smart_198raw,smart_199raw,model_type,tag,flag
dt,,,,,,,,,,,,,,,,,,,,,
2018-03-01,disk_100420,1,83.0,96.0,93.0,100.0,100.0,100.0,23.0,0.0,...,100.0,23.0,0.0,30.0,0.0,0.0,0.0,0,0,0
2018-03-02,disk_100420,1,83.0,96.0,93.0,100.0,100.0,100.0,23.0,0.0,...,100.0,23.0,0.0,30.0,0.0,0.0,0.0,0,0,0
2018-03-03,disk_100420,1,83.0,96.0,93.0,100.0,100.0,100.0,23.0,0.0,...,100.0,23.0,0.0,30.0,0.0,0.0,0.0,0,0,0
2018-03-04,disk_100420,1,83.0,96.0,93.0,100.0,100.0,100.0,23.0,0.0,...,100.0,23.0,0.0,30.0,0.0,0.0,0.0,0,0,0
2018-03-05,disk_100420,1,83.0,96.0,93.0,100.0,100.0,100.0,23.0,0.0,...,100.0,23.0,0.0,30.0,0.0,0.0,0.0,0,0,0
2018-03-06,disk_100420,1,83.0,96.0,93.0,100.0,100.0,100.0,23.0,0.0,...,100.0,23.0,0.0,30.0,0.0,0.0,0.0,0,0,0
2018-03-07,disk_100420,1,83.0,96.0,93.0,100.0,100.0,100.0,23.0,0.0,...,100.0,23.0,0.0,29.0,0.0,0.0,0.0,0,0,0
2018-03-08,disk_100420,1,83.0,96.0,93.0,100.0,100.0,100.0,23.0,0.0,...,100.0,23.0,0.0,30.0,0.0,0.0,0.0,0,0,0
2018-03-09,disk_100420,1,83.0,96.0,93.0,100.0,100.0,100.0,23.0,0.0,...,100.0,23.0,0.0,30.0,0.0,0.0,0.0,0,0,0


In [8]:
def generate_time_series_data(instance, seq_len=20):
    data = []
    tags = []
    
    instance = instance.set_index("dt")
        
    num_of_data = len(instance.index) - (seq_len - 1)

    for i in range(num_of_data):
    
        start_date = instance.index[i]
        end_date = instance.index[i+seq_len-1]
        mask = instance.index >= start_date
        mask &= instance.index <= end_date
    
        tag = instance[mask]["tag"][-1]
    
        time_seq_data = np.array(instance[mask].drop(["serial_number", "model", "tag", "flag"], axis=1))
    
        data.append(time_seq_data)
        tags.append(tag)
    
    return (data, tags)

def sample_by_disk(test_dict, fault_dict):
    sampled_keys = []
    ratio=len(fault_dic)/len(test_dict)
    for key in test_dict.keys():
        if key not in fault_dict:
            if np.random.random_sample() > ratio:
                continue
        sampled_keys += [key]
    return sampled_keys

In [15]:
len(sample_by_disk(test_dict, fault_dic))

196

In [27]:
data_t = []
tag_t = []
sampled_keys = sample_by_disk(test_dict, fault_dic)

with multiprocessing.Pool(processes=8) as p:
    with tqdm(total=len(fault_dic.keys())) as pbar:
        for result in p.imap_unordered(generate_time_series_data, [test_dict[key] for key in fault_dic.keys()]):
            data_t += result[0]
            tag_t += result[1]
            pbar.update()


 74%|███████▍  | 75/101 [00:00<00:00, 302.44it/s]

0
0
0
0
0
0
0
0
0
1
0
0
1
0
4
0
6
0
0
2
7
7
0
8
0
0
0
3
11
11
0
0
0
0
0
0
0
12
0
2
0
2
0
0
0
0
0
0
0
0
0
1
2
0
1
9
12
8
7
11
7
0
0
4
3
1
12
0
8
0
4
0
0
7
4
5


100%|██████████| 101/101 [00:00<00:00, 346.10it/s]


7
0
0
5
4
3
0
0
0
0
0
7
0
12
12
0
0
0
10
2
3
4
9
4
4


In [9]:
keys = test_dict.keys()
chunk_size = 1e4
with multiprocessing.Pool(processes=8) as p:
    with tqdm(total=len(keys)) as pbar:
        data_ = []
        tag_ = []
        count = 0
        for result in p.imap_unordered(generate_time_series_data, [test_dict[key] for key in keys]):
            data_ += result[0]
            tag_ += result[1]
            pbar.update()
            count += 1
            if count >= chunk_size:                   
                np.savez('save_test_%d' % (count//chunk_size), data = data_, tag = tag_)
                count = 0
                data_ = []    
                tag_ = [] 

 10%|▉         | 14859/151178 [02:36<23:54, 95.02it/s]   Process ForkPoolWorker-6:
Process ForkPoolWorker-8:

Process ForkPoolWorker-5:
Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/pool.py", line 135, in worker
    completed += 1
KeyboardInterrupt


KeyboardInterrupt: 

In [21]:
np.savez('save_test', data = data_t, tag = tag_t) 

In [2]:
npzfile = np.load("save_test_1.npz", allow_pickle=True)

In [3]:
print(npzfile["tag"].shape)
npzfile["data"].shape

(116922,)


(116922, 20, 86)

In [5]:
import torch

In [6]:
x = torch.randn(4, 4, 3)

In [56]:
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

class LSTMModel(nn.Module):
    ''' Encodes time-series sequence '''

    def __init__(self, input_size, hidden_size, num_layers, output_size, drop_out=0.5):      
        '''
        : param input_size:     the number of features in the input X
        : param hidden_size:    the number of features in the hidden state h
        : param num_layers:     number of recurrent layers (i.e., 2 means there are
        :                       2 stacked LSTMs)
        '''
        super(LSTMModel, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                            num_layers = num_layers, dropout = drop_out)
        self.classifier = nn.Linear(hidden_size, output_size)
        self.init_weights()

    def forward(self, x_input):
        '''
        : param x_input:               input of shape (seq_len, # in batch, input_size)
        : return lstm_out, hidden:     lstm_out gives all the hidden states in the sequence;
        :                              hidden gives the hidden state and cell state for the last
        :                              element in the sequence 
        '''
        lstm_out, (h, c) = self.lstm(x_input)
        #logit = self.classifier(lstm_out[-1,:,:])

        return lstm_out, (h, c)   

    def init_weights(self):
        '''
        initialize model weight
        '''
        initrange = 0.1
        self.classifier.weight.data.uniform_(-initrange, initrange)
        self.classifier.bias.data.zero_()

    def init_hidden(self, batch_size):
        '''
        initialize hidden state
        : param batch_size:    x_input.shape[1]
        : return:              zeroed hidden state and cell state 
        '''
        return (torch.randn(self.num_layers, batch_size, self.hidden_size),
                torch.randn(self.num_layers, batch_size, self.hidden_size))

In [57]:
model = LSTMModel(input_size=86, hidden_size=200, num_layers=2, output_size=2, drop_out=0.5)

In [16]:
input = npzfile["data"][0:4]

In [26]:
input = input.astype(np.float32)
x_input = torch.from_numpy(input).transpose(1,0)

In [70]:
for i in range(20):
    print(x_input[:,0,:][i])

tensor([ 72.,  96.,  89., 100., 100., 100.,  58.,   0.,  31.,   0.,   0.,   0.,
         72.,  96.,  89., 100., 100., 100.,  58.,   0.,  31.,   0.,   0.,   0.,
         72.,  96.,  89., 100., 100., 100.,  58.,   0.,  31.,   0.,   0.,   0.,
         72.,  96.,  89., 100., 100., 100.,  58.,   0.,  31.,   0.,   0.,   0.,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         82.,  72.,  96.,  89., 100., 100., 100.,  58.,   0.,  31.,   0.,   0.,
          0.,   0.])
tensor([ 72.,  96.,  89., 100., 100., 100.,  58.,   0.,  30.,   0.,   0.,   0.,
         83.,  96.,  89., 100., 100., 100.,  62.,   0.,  31.,   0.,   0.,   0.,
         72.,  96.,  89., 100., 100., 100.,  58.,   0.,  30.,   0.,   0.,   0.,
         83.,  96.,  89., 100., 100., 100.,  62.,   0.,  31.,   0.,   0.,   0.,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,

In [58]:
x_input.shape

torch.Size([20, 4, 86])

In [59]:
output = model(x_input)

In [60]:
lstmoutput = output[0]
h, c = output[1]

In [61]:
lstmoutput.shape

torch.Size([20, 4, 200])

In [63]:
h[-1, :, :]

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan